In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableBranch
from langchain.agents import Tool
from langchain.agents import initialize_agent

In [7]:
import pinecone 

In [8]:
from dotenv import dotenv_values

config = dotenv_values("../.env")

In [9]:
PINECONE_API = config['PINECONE_API']
PINECONE_ENV = config['PINECONE_ENV']
PINECONE_INDEX_NAME = config['PINECONE_INDEX_NAME']

In [ ]:


from pinecone import Pinecone, ServerlessSpec

# Create a Pinecone instance
pc = Pinecone(api_key=PINECONE_API)

# Ensure the index exists or create it
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=1024,  # Replace with the correct dimension for your use case
        metric='cosine',  # Replace with the correct metric for your use case
        spec=ServerlessSpec(
            cloud='aws',
            region=PINECONE_ENV
        )
    )


In [16]:
from FlagEmbedding import FlagModel

model = FlagModel('BAAI/bge-large-en-v1.5', 
                query_instruction_for_retrieval="Represent this sentence for searching relevant passages: ",
                use_fp16=True)

embed = lambda x: model.encode(x).tolist()            

In [ ]:
import os
from pinecone import Pinecone
from langchain.vectorstores import Pinecone

# Initialize Pinecone client
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

# Define your index name
PINECONE_INDEX_NAME = :

# Get the correct Pinecone Index instance
index = pc.Index(PINECONE_INDEX_NAME)  # ✅ Correct instance

# Define the text field name
text_field = "content"  # Adjust based on your data

# Ensure you have an embed function defined
vectorstore = Pinecone(index, embed, text_field)  # ✅ Correct usage


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [ ]:
user_query = '''
Name: Jonathan
Age: 29
Location: New York, USA
Reading Preferences: Fiction, Historical Novels
'''

In [ ]:
llm = ChatOpenAI(
    model_name='gpt-4-1106-preview',
    temperature=0.0
)

In [ ]:
def format_docs_title(docs):
    return "\n\n".join([f"{i+1}. {d.metadata['title']} : {d.page_content}" for i,d in enumerate(docs)])

In [ ]:
pineconeRetreiver = vectorstore.as_retriever(
        search_kwargs={'k' : 10}
)

candidate_books_qa = pineconeRetreiver | format_docs_title

candidate_books_qa.invoke(user_query)

'1. Adventures in Jamestown : In this revised edition, The Liberty Letters® series introduces fictional characters whose courage, ingenuity, and faith shaped events in U.S. history. Through the power of friendship, each story reveals how God works through ordinary teens in extraordinary times.<br />DARING YOUNG WOMEN—IN THE NEW WORLD AND THE OLD-- FIND MORE ADVENTURE THAN THEY BARGAINED FOR<br />After enduring a dangerous voyage to the New World, Abigail discovers that her fight for survival in Jamestown has only begun. When she must face her enemy, an Indian princess called Pocahontas, Abigail uncovers the enemy of her own heart—unforgiveness. In London, England, her friend Elizabeth yearns for adventure, but society’s conventions threaten to crush her dreams. As the girls face their deepest fears, they discover how their choices can change a nation’s---and a young woman’s—destiny.\n\n2. Masada: The Last Fortress : From the Back Cover:<br /><br />"In the year 72 C.E., after a four-yea

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

In [ ]:
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [ ]:
chain = (
    {"candidate_books" : candidate_books_qa, 'user_information' : RunnablePassthrough()}
    | PromptTemplate.from_template(
        """
        You are a recommender. Based on the user's profile and behaviors,
        recommending a suitable book that she will like. Output the title of recommended book.
        
        User Information: 
        {user_information}

        Candidate Books:
        {candidate_books}

        Your response should be a list of comma separated values, eg: `foo, bar, baz`
        """
    )
    | llm
    | output_parser
)

In [ ]:
chain.invoke(user_query)

['Masada: The Last Fortress']